#### Connect to the Database

In [1]:
# Import the required modules
import sqlite3  # SQLite module for database connections and queries
import pandas as pd  # Pandas for handling and displaying query results in DataFrames

# Connect to the SQLite database
# The `connect()` method establishes a connection to the SQLite database file.
# If the database file doesn't exist, it will be created.
conn = sqlite3.connect("Chinook_Sqlite.sqlite")  

# Create a cursor object using the connection
# The cursor is used to execute SQL queries and fetch results.
cursor = conn.cursor()

In [3]:
# Execute a query to retrieve the names of all tables in the SQLite database
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
# Fetch all results of the query, which returns a list of tuples, each containing a table name
tables = cursor.fetchall()

print("Tables and Record Counts:")

# Loop through each table name in the 'tables' list
for table in tables:
    # Extract the table name from the tuple (table[0] because each item in 'tables' is a tuple)
    table_name = table[0]

    # Execute a query to count the number of records (rows) in the current table
    cursor.execute(f"SELECT COUNT(*) FROM {table_name}")
    # Fetch the result, which is a tuple with a single value (the row count)
    record_count = cursor.fetchone()[0]

    # Execute a PRAGMA statement to retrieve column information for the current table
    cursor.execute(f"PRAGMA table_info({table_name})")
    # Fetch the results, which include column details (such as column name, type, etc.)
    columns = cursor.fetchall()

    # Extract the column names from the result, which is a list of tuples (each tuple represents a column)
    column_names = [column[1] for column in columns]  # The second element in each tuple is the column name

    # Print the table name, number of records, and list of columns
    print(f"\nTable: {table_name}")  # Print the table's name
    print(f"Number of Records: {record_count}")  # Print the count of rows in the table
    print(f"Columns: {', '.join(column_names)}")  # Print the column names as a comma-separated list


Tables and Record Counts:

Table: Album
Number of Records: 347
Columns: AlbumId, Title, ArtistId

Table: Artist
Number of Records: 275
Columns: ArtistId, Name

Table: Customer
Number of Records: 59
Columns: CustomerId, FirstName, LastName, Company, Address, City, State, Country, PostalCode, Phone, Fax, Email, SupportRepId

Table: Employee
Number of Records: 8
Columns: EmployeeId, LastName, FirstName, Title, ReportsTo, BirthDate, HireDate, Address, City, State, Country, PostalCode, Phone, Fax, Email

Table: Genre
Number of Records: 25
Columns: GenreId, Name

Table: Invoice
Number of Records: 412
Columns: InvoiceId, CustomerId, InvoiceDate, BillingAddress, BillingCity, BillingState, BillingCountry, BillingPostalCode, Total

Table: InvoiceLine
Number of Records: 2240
Columns: InvoiceLineId, InvoiceId, TrackId, UnitPrice, Quantity

Table: MediaType
Number of Records: 5
Columns: MediaTypeId, Name

Table: Playlist
Number of Records: 18
Columns: PlaylistId, Name

Table: PlaylistTrack
Number o

#### Requirement 1: Complex Joins and CTEs

In [7]:
# SQL query for complex joins and CTEs (Common Table Expressions)
query_complex_joins = """
WITH CustomerTotalSpending AS (  -- Define a CTE named 'CustomerTotalSpending'
    SELECT 
        c.CustomerId,  -- Select CustomerId from the Customer table
        c.FirstName,   -- Select FirstName from the Customer table
        c.LastName,    -- Select LastName from the Customer table
        SUM(il.UnitPrice * il.Quantity) AS TotalSpent  -- Calculate total spending for each customer by multiplying UnitPrice with Quantity in the InvoiceLine table
    FROM Customer c                                         -- Start from the Customer table
    INNER JOIN Invoice i ON c.CustomerId = i.CustomerId     -- Perform an INNER JOIN with the Invoice table using the CustomerId column
    LEFT JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId  -- Perform a LEFT JOIN with the InvoiceLine table to get item details from the InvoiceId
    GROUP BY c.CustomerId          -- Group the data by CustomerId to aggregate the total spending
)
SELECT 
    CustomerId,  -- Select CustomerId from the CTE (CustomerTotalSpending)
    FirstName,   -- Select FirstName from the CTE
    LastName,    -- Select LastName from the CTE
    TotalSpent   -- Select the calculated TotalSpent from the CTE
FROM CustomerTotalSpending  -- From the CTE 'CustomerTotalSpending'
ORDER BY TotalSpent DESC    -- Order the results by TotalSpent in descending order to get the highest spenders at the top
LIMIT 10;                   -- Limit the results to the top 10 customers by total spending
"""

# Execute the SQL query and fetch results into a pandas DataFrame
result_complex_joins = pd.read_sql_query(query_complex_joins, conn)

# Print the results of the query (top 10 customers by total spending)
print(result_complex_joins)


   CustomerId FirstName    LastName  TotalSpent
0           6    Helena        Holý       49.62
1          26   Richard  Cunningham       47.62
2          57      Luis       Rojas       46.62
3          45  Ladislav      Kovács       45.62
4          46      Hugh    O'Reilly       45.62
5          37      Fynn  Zimmermann       43.62
6          24     Frank     Ralston       43.62
7          28     Julia     Barnett       43.62
8          25    Victor     Stevens       42.62
9           7    Astrid      Gruber       42.62


#### Requirement 2: Window Functions for Ranking

In [9]:
# SQL query for window functions to rank products by total sales
query_window_functions = """
SELECT 
    t.Name AS ProductName,           -- Select the name of the product from the Track table
    SUM(il.Quantity) AS TotalSales,  -- Sum up the quantity sold from the InvoiceLine table to get total sales for each product
    RANK() OVER (ORDER BY SUM(il.Quantity) DESC) AS SalesRank  -- Use the RANK() window function to assign a rank based on total sales, in descending order
FROM InvoiceLine il                                            -- From the InvoiceLine table, which contains details of the invoice items
INNER JOIN Track t ON il.TrackId = t.TrackId                   -- Perform an INNER JOIN with the Track table to get the product names
GROUP BY t.Name                                                -- Group the data by product name (Track Name) to calculate the total sales for each product
ORDER BY SalesRank                                             -- Order the results by the SalesRank (ascending order of rank)
LIMIT 10;                                                      -- Limit the results to the top 10 products based on sales rank
"""

# Execute the SQL query and fetch results into a pandas DataFrame
result_window_functions = pd.read_sql_query(query_window_functions, conn)

# Print the results of the query (top 10 products ranked by total sales)
print(result_window_functions)


                 ProductName  TotalSales  SalesRank
0                The Trooper           5          1
1                   Untitled           4          2
2    The Number Of The Beast           4          2
3        Sure Know Something           4          2
4       Hallowed Be Thy Name           4          2
5                   Eruption           4          2
6          Where Eagles Dare           3          7
7  Welcome Home (Sanitarium)           3          7
8             Sweetest Thing           3          7
9                  Surrender           3          7


#### Requirement 3: Indexing and Performance Optimization

In [10]:
import time

# Function to execute query and measure time
def execute_query(query):
    start_time = time.time()  # Start measuring time
    result = pd.read_sql_query(query, conn)  # Execute query and store the result
    end_time = time.time()  # End measuring time
    execution_time = end_time - start_time  # Calculate execution time
    return result, execution_time

# SQL query to list total sales for each customer (without indexing)
query_without_index = """
SELECT 
    c.CustomerId,
    c.FirstName || ' ' || c.LastName AS CustomerName,  -- Combine first and last name to display full customer name
    SUM(il.UnitPrice * il.Quantity) AS TotalSpent      -- Calculate total spending by multiplying price and quantity in InvoiceLine table
FROM Customer c
INNER JOIN Invoice i ON c.CustomerId = i.CustomerId      -- Join Customer table with Invoice table based on CustomerId
INNER JOIN InvoiceLine il ON i.InvoiceId = il.InvoiceId  -- Join Invoice table with InvoiceLine table based on InvoiceId
GROUP BY c.CustomerId                                    -- Group by customer to calculate total spending per customer
ORDER BY TotalSpent DESC;                                -- Order the results by total spending, in descending order
"""

# Step 1: Execute the query without indexing
print("Executing query without indexing...")
result_without_index, execution_time_without_index = execute_query(query_without_index)  # Execute the query
print(f"Execution Time Without Indexing: {execution_time_without_index:.6f} seconds")  # Print execution time

# Display the top results without indexing
print("\nTop Customers by Total Sales (Without Indexing):")
print(result_without_index.head())  # Display the first few rows of the result

# Step 2: Create an index on the CustomerId column in the Invoice table
print("\nCreating index on CustomerId column in Invoice table...")
create_index_query = "CREATE INDEX IF NOT EXISTS idx_invoice_customerid ON Invoice (CustomerId);"  # Create index query
conn.execute(create_index_query)  # Execute the index creation query
conn.commit()  # Commit the changes to the database
print("Index created successfully.")  # Print success message

# Step 3: Execute the query with indexing
print("\nExecuting query with indexing...")
result_with_index, execution_time_with_index = execute_query(query_without_index)  # Execute the same query again with the index
print(f"Execution Time With Indexing: {execution_time_with_index:.6f} seconds")  # Print execution time with indexing

# Step 4: Compare the execution times
print("\n--- Query Performance Comparison ---")
print(f"Execution Time Without Indexing: {execution_time_without_index:.6f} seconds")  # Print comparison
print(f"Execution Time With Indexing: {execution_time_with_index:.6f} seconds")  # Print comparison
print("\n------------------------------------")

#Display the results (Top 5 customers)
print("\nTop 5 Customers by Total Sales (With Indexing):")
print(result_with_index.head())  # Display the top 5 customers with indexing


Executing query without indexing...
Execution Time Without Indexing: 0.013862 seconds

Top Customers by Total Sales (Without Indexing):
   CustomerId        CustomerName  TotalSpent
0           6         Helena Holý       49.62
1          26  Richard Cunningham       47.62
2          57          Luis Rojas       46.62
3          46       Hugh O'Reilly       45.62
4          45     Ladislav Kovács       45.62

Creating index on CustomerId column in Invoice table...
Index created successfully.

Executing query with indexing...
Execution Time With Indexing: 0.009097 seconds

--- Query Performance Comparison ---
Execution Time Without Indexing: 0.013862 seconds
Execution Time With Indexing: 0.009097 seconds

------------------------------------

Top 5 Customers by Total Sales (With Indexing):
   CustomerId        CustomerName  TotalSpent
0           6         Helena Holý       49.62
1          26  Richard Cunningham       47.62
2          57          Luis Rojas       46.62
3          46   

In [11]:
# Close the database connection
conn.close()